In [1]:
from langchain.agents import create_sql_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase

llm = ChatOpenAI(
    temperature=0.1,
    model_name="gpt-4-1106-preview",
)
db = SQLDatabase.from_uri("sqlite:///movies.sqlite")

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
)

agent.invoke(
    "Give me the movies that have the highest votes but the lowest budgets and give me the name of their directors also include their gross revenue."
)



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with ``


directors, movies
Invoking: `sql_db_schema` with `movies,directors`


Error: table_names {'movies,directors'} not found in database
Invoking: `sql_db_schema` with `movies`



CREATE TABLE movies (
	id INTEGER, 
	original_title VARCHAR, 
	budget INTEGER, 
	popularity INTEGER, 
	release_date TEXT, 
	revenue INTEGER, 
	title TEXT, 
	vote_average REAL, 
	vote_count INTEGER, 
	overview TEXT, 
	tagline TEXT, 
	uid INTEGER, 
	director_id INTEGER DEFAULT 0 NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows from movies table:
id	original_title	budget	popularity	release_date	revenue	title	vote_average	vote_count	overview	tagline	uid	director_id
43597	Avatar	237000000	150	2009-12-10	2787965087	Avatar	7.2	11800	In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but 	Enter the World of Pandora.	19995	4762
43598	Pirates of the Caribbean: At World's End	300000000	139	2007-05-19	961000

{'input': 'Give me the movies that have the highest votes but the lowest budgets and give me the name of their directors also include their gross revenue.',
 'output': 'Here are movies that have the highest votes but the lowest budgets, along with the name of their directors and their gross revenue:\n\n1. \'Stiff Upper Lips\' directed by Gary Sinyor, with a budget of $0, a vote average of 10.0, and a revenue of $0.\n2. \'Dancer, Texas Pop. 81\' directed by Tim McCanlies, with a budget of $0, a vote average of 10.0, and a revenue of $565,592.\n3. \'Sardaarji\' directed by Rohit Jugraj, with a budget of $0, a vote average of 9.5, and a revenue of $0.\n4. "One Man\'s Hero" directed by Lance Hool, with a budget of $0, a vote average of 9.3, and a revenue of $0.\n5. \'There Goes My Baby\' directed by Floyd Mutrux, with a budget of $10,500,000, a vote average of 8.5, and a revenue of $123,509.\n6. \'The Shawshank Redemption\' directed by Frank Darabont, with a budget of $25,000,000, a vote a